In [ ]:
import cv2
import numpy as np
import os

# ---------------------------
# 파라미터 설정
# ---------------------------
width, height = 7, 10  # 체커보드 내부 코너 수
square_size = 0.020  # 한 칸 길이 (미터)
num_images = 50      # 저장할 이미지 수
save_dir = "C:/cal_cam"
os.makedirs(save_dir, exist_ok=True)

# ---------------------------
# 카메라 열기
# ---------------------------
cap = cv2.VideoCapture(1)

#cap.set(cv2.CAP_PROP_EXPOSURE, -6)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)  # 해상도 높게 설정 (옵션)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

print("[INFO] 실시간 체커보드 인식 및 캘리브레이션 준비 시작...")

# ---------------------------
# 캡처된 데이터 저장
# ---------------------------
objpoints = []  # 3D points in real world
imgpoints = []  # 2D points in image plane

# 체커보드 월드 좌표
objp = np.zeros((height * width, 3), np.float32)
objp[:, :2] = np.mgrid[0:width, 0:height].T.reshape(-1, 2) * square_size

capture_count = 0

# ---------------------------
# 실시간 루프
# ---------------------------
while capture_count < num_images:
    ret, frame = cap.read()
    if not ret:
        print("[ERROR] 카메라 연결 실패")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (width, height), None)

    if ret:
        # 코너 정밀 조정
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), 
                                    (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))

        # 코너 그리기
        cv2.drawChessboardCorners(frame, (width, height), corners2, ret)

        # 코너가 잘 잡혔을 때만 자동 저장
        objpoints.append(objp)
        imgpoints.append(corners2)
        filename = os.path.join(save_dir, f"calib_{capture_count:03d}.png")
        cv2.imwrite(filename, frame)
        print(f"[INFO] 저장됨: {filename}")
        capture_count += 1

    # 상태 표시
    cv2.putText(frame, f"Captured: {capture_count}/{num_images}", (20, 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)

    cv2.imshow("Calibration Capture", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("[INFO] 사용자 중단")
        break

cap.release()
cv2.destroyAllWindows()

# ---------------------------
# 저장한 이미지로 캘리브레이션 수행
# ---------------------------
if len(objpoints) >= 5:  # 최소 5장 이상 권장
    print("[INFO] 캘리브레이션 시작...")

    # 마지막 gray.shape로 해도 되고, frame.shape 사용 가능
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
        objpoints, imgpoints, gray.shape[::-1], None, None
    )

    print(f"\n[RESULT] RMS 재투영 오차: {ret:.4f}")
    print(f"[RESULT] 카메라 행렬 (mtx):\n{mtx}")
    print(f"[RESULT] 왜곡 계수 (dist):\n{dist}")

    # 결과 저장
    np.save(os.path.join(save_dir, "calibration_matrix.npy"), mtx)
    np.save(os.path.join(save_dir, "distortion_coefficients.npy"), dist)
    print(f"[INFO] 결과가 {save_dir} 에 저장되었습니다.")
else:
    print("[ERROR] 충분한 체커보드 데이터가 없습니다. 다시 시도하세요.")


[INFO] 실시간 체커보드 인식 및 캘리브레이션 준비 시작...
[INFO] 저장됨: C:/cal_cam\calib_000.png
[INFO] 저장됨: C:/cal_cam\calib_001.png
[INFO] 저장됨: C:/cal_cam\calib_002.png
[INFO] 저장됨: C:/cal_cam\calib_003.png
[INFO] 저장됨: C:/cal_cam\calib_004.png
[INFO] 저장됨: C:/cal_cam\calib_005.png
[INFO] 저장됨: C:/cal_cam\calib_006.png
[INFO] 저장됨: C:/cal_cam\calib_007.png
[INFO] 저장됨: C:/cal_cam\calib_008.png
[INFO] 저장됨: C:/cal_cam\calib_009.png
[INFO] 저장됨: C:/cal_cam\calib_010.png
[INFO] 저장됨: C:/cal_cam\calib_011.png
[INFO] 저장됨: C:/cal_cam\calib_012.png
[INFO] 저장됨: C:/cal_cam\calib_013.png
[INFO] 저장됨: C:/cal_cam\calib_014.png
[INFO] 저장됨: C:/cal_cam\calib_015.png
[INFO] 저장됨: C:/cal_cam\calib_016.png
[INFO] 저장됨: C:/cal_cam\calib_017.png
[INFO] 저장됨: C:/cal_cam\calib_018.png
[INFO] 저장됨: C:/cal_cam\calib_019.png
[INFO] 저장됨: C:/cal_cam\calib_020.png
[INFO] 저장됨: C:/cal_cam\calib_021.png
[INFO] 저장됨: C:/cal_cam\calib_022.png
[INFO] 저장됨: C:/cal_cam\calib_023.png
[INFO] 저장됨: C:/cal_cam\calib_024.png
[INFO] 저장됨: C:/cal_cam\calib_025.png
[